In [ ]:
using Images, Colors, Distributions

function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho) #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto)#el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 1:alto, x in 1:ancho]
end

function moneda(N)
    ceil(Int64, round(rand(Uniform(-N-0.499, N+0.499))))
end

function paleta(mi_foto, paso, tamaño)
    aesthetics = load(mi_foto)
    alto, ancho =size(aesthetics)
    x0 = ceil(Int64, rand(Uniform(0,ancho)))
    #x0 = ceil(Int64, ancho/2)
    y0 = ceil(Int64, rand(Uniform(0,alto)))
    #y0 = ceil(Int64, alto/2)
    paleta = zeros(RGB{N0f8}, tamaño)
    for i = 1:length(paleta)
        paleta[i] = aesthetics[mod(y0,alto)+1,mod(x0,ancho)+1]
        x0 = x0 + moneda(paso)
        y0 = y0 + moneda(paso)
    end
    return paleta
end

# Iteración de funciones elípticas

En este ejemplo vamos a dibujar la dinámica de algunas variaciones de la función $\wp$ de Weierstrass. Para evaluar la función $\wp$ vamos a usar una aproximación numérica con ayuda de la función de Lateés asociada.

Una reticula es un subgrupo aditivo de $\mathbb{C}$ de dos generadores. Podemos escribirlo como $$ \Lambda (\omega_1 , \omega_2) =  \{ n \omega_1  + m \omega_2 : n, m \in \mathbb{Z} \}, $$ donde $\omega_1, \omega_2 \in \mathbb{C}$ son generadores. Al toro cociente lo denotaremos $\mathbb{T}(\omega_1, \omega_2) = \mathbb{C} / \Lambda(\omega_1, \omega_2)$.

La función $\wp: \mathbb{C} \rightarrow \hat{\mathbb{C}}$ es, salvo elección de cartas, la proyección del plano complejo al cociente $\mathbb{T}(\omega_1, \omega_2) / \iota \simeq \hat{\mathbb{C}}$, donde $\iota$ es la involución del toro inducida por $z \mapsto -z$. Una formula para esta función es $$ \wp(z) = \frac{1}{z^2} + \sum_{\omega \in \Lambda \setminus 0} \left( \frac{1}{(z - \omega)^2} - \frac{1}{\omega^2} \right).$$

Nos recordamos que la función de Lateés asociada a una retícula es la funcion racional $L: \hat{\mathbb{C}} \rightarrow \hat{\mathbb{C}}$ que esta semiconjugada mediante $\wp$ a un homomorfismo de la retícula de la forma $z \mapsto n z$ con $n \in \mathbb{Z}$. En otras palabras, si $n: z \mapsto n z$ es el homomorfismo de la retícula $\Lambda$, tenemos la ecuación dinámica dada por la semiconjugación$$ \wp(n^k z) = L^k \circ \wp (z).$$ 

En una vecindad del origen podemos aproximar la función $\wp$ por un truncamiento de su serie de Laurent $$\wp(z) \approx \frac{1}{z^2} + \frac{G_2}{20} z^2 + \frac{G_3}{28} z^4 =: Q(z),$$ donde $G_2$ y $G_3$ son los *invariantes modulares* de la retícula, dados por $$G_2(\Lambda) = 60 \sum_{\omega \in \Lambda \setminus 0}\left(\frac{1}{\omega^4}\right), \hspace{2em} G_3(\Lambda) = 140 \sum_{\omega \in \Lambda \setminus 0}\left(\frac{1}{\omega^6}\right) .$$

Aplicando la inversa de $n$ a la ecuación dinámica y sustituyendo el trucnamiento de la serie de Laurent de $\wp$ obtenemos $$ \wp(z) \approx L^k \circ Q \left( \frac{z}{n^k} \right).$$

En la siguiente función ``WP`` usamos $n=2$ y la función de Lateés correspondiente es $$ L(z) = \cfrac{(6 z^2 - \frac{G_2}{2})^2}{4(4 z^3 - z G_2 - G_3)} - 2 z.$$

In [ ]:
function WP(Z, G2, G3, K)
    W = Z/(2.0^K)
    W = 1.0/(W*W) + (G2*W*W)/20.0 + (G3*W*W*W*W)/28.0
    for i = 1:K
        W = -2.0*W + (6.0*W*W - 0.5*G2)*(6.0*W*W - 0.5*G2)/(4.0*(4.0*W*W*W - G2*W - G3))
    end
    return W
end

Para empezar, sabemos que $\wp$ tiene un polo de orden dos en cada punto de la retícula $\Lambda$ y que sus puntos críticos son los *periodos medios* $$\left\{ \frac{n \omega_1 + m \omega_2}{2} : n, m \in \mathbb{Z} \right\}$$ donde $ \omega_1 , \omega_2$ son generadores de $\Lambda$.

## Primer dibujito elíptico: iteración de $1 / \wp$

La función elíptica $1 / \wp$ tiene un cero de orden dos en el origen, es decir, un punto fijo superatractor en cero. El dibujito que haremos será la trampa de este atractor.

In [ ]:
function trampa_eliptica_Pinv(Z, G2, G3, K, paleta, eps=0.05, TOPE=1000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < eps
            n = 1 + mod(cont, length(paleta))
            return paleta[n]
        end
        Z = 1.0/WP(Z, G2, G3, K)
        cont += 1
    end
    return RGB{Float32}(0.0, 0.0, 0.0) #color negro
end

function Pinv_atractor(G2, G3, K, paleta, eps=0.05, TOPE=1000,radio=5.0,centro=0.0,alto=1000,ancho=1000)
    broadcast(Z -> trampa_eliptica_Pinv(Z, G2, G3, K, paleta,eps,TOPE), muestra(radio,centro,alto,ancho))
end

In [ ]:
mis_colores = paleta("aesthetic_8.jpg", 8, 1000)

In [ ]:
Pinv_atractor(0.0, 0.1im, 5, mis_colores, 0.2, 2000)